In [5]:
# Import the dependencies
import pandas as pd
import gmaps
import requests

# Import the API key
from config import g_key

In [6]:
# Store the CSV saved from part one into a dataframe
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Nurota,40.5614,65.6886,48.76,41,8,4.83,UZ,2022-11-12 07:26:38
1,1,San Patricio,28.0170,-97.5169,52.84,64,100,20.11,US,2022-11-12 07:26:39
2,2,Ushuaia,-54.8000,-68.3000,38.86,75,0,5.75,AR,2022-11-12 07:24:28
3,3,Sanmenxia,34.7747,111.1814,55.56,33,99,21.18,CN,2022-11-12 07:26:39
4,4,Turaif,31.6725,38.6637,64.65,63,0,20.71,SA,2022-11-12 07:26:39


In [7]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Configure gmaps to use the Google API key
gmaps.configure(api_key = g_key)

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 300, point_radius = 4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"] 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = clouds ,dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"] 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = wind_speed ,dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add minimum and maximum temperature values
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,Bandarbeyla,9.4942,50.8122,82.58,68,24,17.13,SO,2022-11-12 07:25:20
11,11,Nelson Bay,-32.7167,152.1500,75.25,79,100,10.00,AU,2022-11-12 07:21:47
14,14,Alappuzha,9.4900,76.3264,86.00,73,100,8.08,IN,2022-11-12 07:26:42
15,15,Amla,21.9333,78.1167,78.49,38,25,5.26,IN,2022-11-12 07:26:42
21,21,Yeppoon,-23.1333,150.7333,82.18,55,0,13.13,AU,2022-11-12 07:26:44
22,22,Vaini,-21.2000,-175.2000,79.00,84,10,5.28,TO,2022-11-12 07:26:44
23,23,East London,-33.0153,27.9116,85.14,51,0,8.05,ZA,2022-11-12 07:26:44
30,30,Capuluan,13.8147,122.5181,88.09,66,96,7.49,PH,2022-11-12 07:23:21
37,37,Ouadda,8.0777,22.4007,81.28,29,1,7.36,CF,2022-11-12 07:26:48
42,42,Katobu,-4.9408,122.5276,83.84,77,100,1.68,ID,2022-11-12 07:26:50


In [13]:
# Check for null values
preferred_cities_df.count()

City_ID       166
City          166
Lat           166
Lng           166
Max Temp      166
Humidity      166
Cloudiness    166
Wind Speed    166
Country       165
Date          166
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Bandarbeyla,SO,82.58,9.4942,50.8122,
11,Nelson Bay,AU,75.25,-32.7167,152.1500,
14,Alappuzha,IN,86.00,9.4900,76.3264,
15,Amla,IN,78.49,21.9333,78.1167,
21,Yeppoon,AU,82.18,-23.1333,150.7333,
22,Vaini,TO,79.00,-21.2000,-175.2000,
23,East London,ZA,85.14,-33.0153,27.9116,
30,Capuluan,PH,88.09,13.8147,122.5181,
37,Ouadda,CF,81.28,8.0777,22.4007,
42,Katobu,ID,83.84,-4.9408,122.5276,


In [15]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [19]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
# hotels

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


{'html_attributions': [],
 'next_page_token': 'AW30NDxwLJd6z-8r9gO1LUvrmmS6ABcj5h8P2PJjp7xpLqF0iDtds7On3kJAjvUY7zshGIzSjdM8b25KLv9U5EtZw-XUoWZDnnD2YUFBS_IajaUH_paOKj9M52TLpgohQBmplgDLH2Wbgxv--y0dIXUgmcmbWS51fc_9HOGDZHkFXbM5VRQ24SiYvU-Np-owErVxPSnDW_os8YEn0_gycxHTYJDO0k_bzJJAaAcbODQR6GXCa0uXPsVX8OBnh_iy1ZMMaXYyhhjZjMTWfniYUmIE8yJlsWKqi4ejPf970BtzI39iloSyberB621ModcNAVWe0umJsqYMxM2u9ypooaPg15gG2veNgFvfWDjyRXe_gIdKndJHVYxLYWTfOOWx3KMkO6Vwne-XjKpE48ErAE9uFH14TI3muOryyERMyHzacJsF2r804zdTqxf8',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -0.8968026999999997, 'lng': 119.8728966},
    'viewport': {'northeast': {'lat': -0.8953298697084979,
      'lng': 119.8742711302915},
     'southwest': {'lat': -0.898027830291502, 'lng': 119.8715731697085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_

In [21]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Bandarbeyla,SO,82.58,9.4942,50.8122,JABIR HOTEL
11,Nelson Bay,AU,75.25,-32.7167,152.1500,Mantra Nelson Bay
14,Alappuzha,IN,86.00,9.4900,76.3264,Panoramic Sea Resort
15,Amla,IN,78.49,21.9333,78.1167,L l ganeshkar
21,Yeppoon,AU,82.18,-23.1333,150.7333,Villa Mar Colina


In [20]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))